<a href="https://colab.research.google.com/github/Hettieboo/AlgorithmicTradingProject/blob/main/Final_WorkingScript_LotsAutomation_2D%263D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

# Upload your Excel file (it will prompt you to pick it from your computer)
uploaded = files.upload()

Saving calculateur convelio 23000 24.09.25 (1).xlsx to calculateur convelio 23000 24.09.25 (1).xlsx


In [ ]:
# COMPLETE ENHANCED AUCTION DIMENSION PROCESSOR - SHIPPING-READY VERSION
# =======================================================================

# STEP 1: Install packages and imports
!pip install openpyxl

import pandas as pd
import re
import numpy as np
from typing import List, Dict, Optional
from google.colab import files

# STEP 2: Enhanced Main Extraction Class
class AuctionDimensionExtractor:
    def __init__(self):
        self.multiples_keywords = {
            "paire": 2, "deux": 2, "trois": 3, "quatre": 4,
            "cinq": 5, "six": 6, "sept": 7, "huit": 8,
            "neuf": 9, "dix": 10, "onze": 11, "douze": 12,
            "treize": 13, "quatorze": 14, "quinze": 15,
            "seize": 16, "dix-sept": 17, "dix-huit": 18,
            "dix-neuf": 19, "vingt": 20
        }

        self.dimension_patterns = {
            'H': re.compile(r'H\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'L': re.compile(r'L\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'P': re.compile(r'P\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'Diameter': re.compile(r'Ø\s*[:\s]*(\d+(?:[.,]\d+)?)', re.IGNORECASE),
            'TwoD': re.compile(r'(\d+(?:[.,]\d+)?)\s*[×x]\s*(\d+(?:[.,]\d+)?)\s*cm', re.IGNORECASE)
        }

        self.complete_dimension_pattern = re.compile(
            r'H\s*[:\s]*(\d+(?:[.,]\d+)?)\s*[×x]\s*(?:L\s*[:\s]*(\d+(?:[.,]\d+)?)\s*[×x]\s*P\s*[:\s]*(\d+(?:[.,]\d+)?)|Ø\s*[:\s]*(\d+(?:[.,]\d+)?))',
            re.IGNORECASE
        )

    def normalize_number(self, num_str: str) -> Optional[float]:
        if not num_str:
            return None
        try:
            return float(num_str.replace(',', '.'))
        except (ValueError, AttributeError):
            return None

    def detect_multiples(self, text: str) -> int:
        if not isinstance(text, str):
            return 1
        text_lower = text.lower()
        for word, count in self.multiples_keywords.items():
            if word in text_lower:
                return count
        digit_pattern = re.search(r'ensemble\s+de\s+(\d+)', text_lower)
        if digit_pattern:
            try:
                return int(digit_pattern.group(1))
            except ValueError:
                pass
        return 1

    # UPDATED 2D/3D detection to avoid misclassifying furniture
    def detect_item_type(self, typeset_text: str) -> str:
        if not isinstance(typeset_text, str):
            return '3D'
        text_lower = typeset_text.lower()

        # Only check first 300 chars (usually description)
        preview_text = text_lower[:300]

        artwork_keywords = [
            'huile', 'gouache', 'aquarelle', 'acrylique', 'pastel', 'crayon',
            'dessin', 'gravure', 'lithographie', 'sérigraphie', 'estampe',
            'papier', 'toile', 'canvas', 'carton', 'technique mixte',
            'oil', 'watercolor', 'acrylic', 'drawing', 'print', 'painting',
            'encre', 'fusain', 'sanguine', 'collage', 'mixed media'
        ]

        # Ignore keywords if in bibliography/provenance context
        ignore_terms = ['provenance', 'bibliographie', 'catalogue', 'album', 'exposition', 'collection']
        if any(keyword in preview_text for keyword in artwork_keywords) and not any(term in preview_text for term in ignore_terms):
            return '2D'

        # Check dimensions: H+L without P or Diameter
        dims = self.extract_all_dimensions(typeset_text)
        for d in dims:
            H, L, P, Dia = d['H'], d['L'], d['P'], d['Diameter']
            if (H and L) and not (P or Dia):
                return '2D'

        return '3D'

    def extract_dimensions_from_segment(self, segment: str) -> List[Dict[str, Optional[float]]]:
        dimensions = []
        complete_matches = self.complete_dimension_pattern.findall(segment)
        if complete_matches:
            for match in complete_matches:
                h, l, p, diameter = match
                dimensions.append({
                    'H': self.normalize_number(h),
                    'L': self.normalize_number(l) if l else None,
                    'P': self.normalize_number(p) if p else None,
                    'Diameter': self.normalize_number(diameter) if diameter else None
                })
        else:
            two_d_match = self.dimension_patterns['TwoD'].search(segment)
            if two_d_match:
                h_val = self.normalize_number(two_d_match.group(1))
                l_val = self.normalize_number(two_d_match.group(2))
                dimensions.append({'H': h_val, 'L': l_val, 'P': None, 'Diameter': None})
            else:
                h_matches = [self.normalize_number(m) for m in self.dimension_patterns['H'].findall(segment)]
                l_matches = [self.normalize_number(m) for m in self.dimension_patterns['L'].findall(segment)]
                p_matches = [self.normalize_number(m) for m in self.dimension_patterns['P'].findall(segment)]
                d_matches = [self.normalize_number(m) for m in self.dimension_patterns['Diameter'].findall(segment)]
                max_len = max(len(h_matches), len(l_matches), len(p_matches), len(d_matches), 1)
                for i in range(max_len):
                    dimensions.append({
                        'H': h_matches[i] if i < len(h_matches) else None,
                        'L': l_matches[i] if i < len(l_matches) else None,
                        'P': p_matches[i] if i < len(p_matches) else None,
                        'Diameter': d_matches[i] if i < len(d_matches) else None
                    })
        dimensions = [d for d in dimensions if any(v is not None for v in d.values())]
        return dimensions if dimensions else [{'H': None, 'L': None, 'P': None, 'Diameter': None}]

    def extract_all_dimensions(self, text: str) -> List[Dict[str, Optional[float]]]:
        if not isinstance(text, str):
            return [{'H': None, 'L': None, 'P': None, 'Diameter': None}]
        text = text.replace('\n', ' ').replace('\xa0', ' ').strip()
        all_dimensions = []
        segments = re.split(r'[;\n]', text)
        for segment in segments:
            segment = segment.strip()
            if segment:
                dims = self.extract_dimensions_from_segment(segment)
                all_dimensions.extend(dims)
        return all_dimensions if all_dimensions else [{'H': None, 'L': None, 'P': None, 'Diameter': None}]

    def process_dataframe(self, df: pd.DataFrame, typeset_column: str = 'TYPESET') -> pd.DataFrame:
        print(f"Processing {len(df)} rows...")
        result_rows = []
        for idx, row in df.iterrows():
            if idx % 100 == 0:
                print(f"Progress: {idx}/{len(df)} rows processed")
            typeset_text = str(row[typeset_column]) if pd.notna(row[typeset_column]) else ""
            num_items = self.detect_multiples(typeset_text)
            item_type = self.detect_item_type(typeset_text)
            dimensions_list = self.extract_all_dimensions(typeset_text)
            if len(dimensions_list) == 1 and num_items > 1:
                dimensions_list = dimensions_list * num_items
            elif len(dimensions_list) > 1 and num_items > len(dimensions_list):
                cycles_needed = (num_items + len(dimensions_list) - 1) // len(dimensions_list)
                dimensions_list = (dimensions_list * cycles_needed)[:num_items]
            for dims in dimensions_list:
                new_row = row.copy()
                new_row['H'] = dims['H']
                new_row['L'] = dims['L']
                new_row['P'] = dims['P']
                new_row['Diameter'] = dims['Diameter']
                new_row['ITEM_COUNT'] = num_items
                new_row['ITEM_TYPE'] = item_type
                if item_type == '2D':
                    h_val = dims['H'] if dims['H'] is not None else 0
                    l_val = dims['L'] if dims['L'] is not None else 0
                    if h_val and l_val:
                        new_row['L'] = max(h_val, l_val)
                        new_row['H'] = min(h_val, l_val)
                    elif h_val:
                        new_row['L'] = h_val
                    elif l_val:
                        new_row['L'] = l_val
                    new_row['D'] = 5.0
                else:
                    if dims['Diameter'] is not None:
                        new_row['D'] = dims['Diameter']
                    elif dims['P'] is not None:
                        new_row['D'] = dims['P']
                    elif dims['L'] is not None:
                        new_row['D'] = dims['L']
                    else:
                        new_row['D'] = None
                result_rows.append(new_row)
        df_final = pd.DataFrame(result_rows)
        if 'ITEM_COUNT' not in df_final.columns:
            df_final['ITEM_COUNT'] = 1
        if 'ITEM_TYPE' not in df_final.columns:
            df_final['ITEM_TYPE'] = '3D'
        print(f"Completed! Expanded {len(df)} rows to {len(df_final)} rows")
        return df_final

# STEP 3: Enhanced Shipping Standardization Function
def standardize_for_shipping(df_processed):
    print("Standardizing for shipping...")
    df = df_processed.copy()
    df['CONVERSION_LOG'] = ''
    for idx, row in df.iterrows():
        conversions = []
        item_type = row.get('ITEM_TYPE', '3D')
        if item_type == '2D':
            conversions.append("2D: L=max(H,L), D=5")
        else:
            if pd.notna(row['Diameter']):
                if pd.isna(row['L']) or row['L'] != row['Diameter']:
                    df.at[idx, 'L'] = row['Diameter']
                    conversions.append("L=Ø")
                if row['D'] != row['Diameter']:
                    df.at[idx, 'D'] = row['Diameter']
                    conversions.append("D=Ø")
            elif pd.notna(row['P']) and row['D'] != row['P']:
                df.at[idx, 'D'] = row['P']
                conversions.append("D=P")
        if conversions:
            df.at[idx, 'CONVERSION_LOG'] = "; ".join(conversions)
    print("Shipping-ready standardization done.")
    return df

# STEP 4: Analysis Function
def analyze_2d_3d_split(df):
    print("\n2D/3D Analysis:")
    if 'ITEM_TYPE' in df.columns:
        type_counts = df['ITEM_TYPE'].value_counts()
        print(f"2D items: {type_counts.get('2D', 0)}")
        print(f"3D items: {type_counts.get('3D', 0)}")
        two_d_items = df[df['ITEM_TYPE'] == '2D'].head(3)
        if len(two_d_items) > 0:
            print("\nSample 2D items:")
            for _, row in two_d_items.iterrows():
                print(f"LOT {row.get('LOT', 'N/A')}: H={row['H']}, L={row['L']}, D={row['D']}")
        three_d_items = df[df['ITEM_TYPE'] == '3D'].head(3)
        if len(three_d_items) > 0:
            print("\nSample 3D items:")
            for _, row in three_d_items.iterrows():
                print(f"LOT {row.get('LOT', 'N/A')}: H={row['H']}, L={row['L']}, D={row['D']}, Ø={row.get('Diameter', 'N/A')}")

# MAIN EXECUTION
print("Starting Complete Auction Dimension Processing...")

df = pd.read_excel('calculateur convelio 23000 24.09.25 (1).xlsx')
print(f"Loaded {len(df)} rows and {len(df.columns)} columns")

extractor = AuctionDimensionExtractor()
df_processed = extractor.process_dataframe(df)

analyze_2d_3d_split(df_processed)

df_shipping_ready = standardize_for_shipping(df_processed)

# Reorder columns: original first, then first calculated, then remaining calculated
original_cols = ['SALENO', 'LOT', 'TYPESET', 'LOW', 'HIGH']
first_calculated_cols = ['ITEM_TYPE', 'ITEM_COUNT']
other_calculated_cols = ['H', 'L', 'P', 'Diameter', 'D', 'CONVERSION_LOG']
final_columns = original_cols + first_calculated_cols + other_calculated_cols
remaining_cols = [c for c in df_shipping_ready.columns if c not in final_columns]
df_shipping_ready = df_shipping_ready[final_columns + remaining_cols]

filename = 'auction_dimensions_shipping_ready_enhanced.xlsx'
df_shipping_ready.to_excel(filename, index=False)
files.download(filename)

print(f"All done! Enhanced shipping-ready file saved as '{filename}'")


Starting Complete Auction Dimension Processing...
Loaded 208 rows and 5 columns
Processing 208 rows...
Progress: 0/208 rows processed
Progress: 100/208 rows processed
Progress: 200/208 rows processed
Completed! Expanded 208 rows to 963 rows

2D/3D Analysis:
2D items: 962
3D items: 1

Sample 2D items:
LOT 1: H=44.5, L=65.0, D=5.0
LOT 1: H=44.5, L=65.0, D=5.0
LOT 1: H=44.5, L=65.0, D=5.0

Sample 3D items:
LOT 152: H=nan, L=nan, D=25.0, Ø=25.0
Standardizing for shipping...
Shipping-ready standardization done.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All done! Enhanced shipping-ready file saved as 'auction_dimensions_shipping_ready_enhanced.xlsx'
